# Import Libraries

In [3]:
# Libraries
import pandas as pd
import numpy as np
import requests 
import re
import joblib

# Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# BeautifulSoup
from bs4 import BeautifulSoup as bs

# Permanently changes the pandas display settings
pd.set_option('display.max_columns', None)

# Suppress FutureWarning messages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Functions

In [4]:
# Function to remove leading zero from a two-digit number
def drop_lead_zero(grade_number):
    
    # Define a regex pattern to remove any leading zeros from the two-digit results
    pattern = r'\b0(\d)\b'

    # Use re.sub to replace matched pattern with second digit only
    grade_number = re.sub(pattern, r'\1', grade_number)

    return grade_number

In [5]:
# Function to display School Type column, along with new Grade columns
def print_grade_cols():

    # Find unique values for School Type
    unique_School_Type = dfx['School Type'].unique()

    # Create a temporary dataframe to assist in printing desired information, simply
    temp_dfx = pd.DataFrame(columns=dfx.columns)

    # For each School Type, append row of first instance to temporary dataframe
    for type in unique_School_Type:
        row = dfx[dfx['School Type'] == type].iloc[0]
        temp_dfx = temp_dfx.append(row, ignore_index=True)
        
    # Print out only School Type and Grade columns
    display(pd.concat([temp_dfx.iloc[:, 11], temp_dfx.iloc[:,16:]], axis = 1))

# Web Scrape

In [205]:
# Launch Browser 
# Setup the Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [228]:
# Initialize DataFrame
# Specify column names to use with dataframe
columns = ['School Name', 'Full Address', 'Street Address', 'City', 'State', 'Zip Code', 'Country', 'School Type', 'Website', 'Description']

# Initialize DataFrame
df = pd.DataFrame(columns=columns)

print(df)

Empty DataFrame
Columns: [School Name, Full Address, Street Address, City, State, Zip Code, Country, School Type, Website, Description]
Index: []


In [229]:
# Scrape data from each state/province page 
# Create list of States & Provinces in the USA & Canada to loop over
state_province = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'ON', 'PE', 'QC', 'SK']
province = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'ON', 'PE', 'QC', 'SK']

for state in state_province:

    if state in province: 
        co_code = 'CA'

    else:
        co_code = 'US'

    # Create link for each state/province 
    url = f'https://www.adventisteducation.org/schools?stateProvince={state}%3B{co_code}'

    # Open page
    driver.get(url)

    # Wait for the element to become clickable
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'Schools_searchResults__')]"))
    )
    # Once the element is clickable, get the outer HTML
    html_content = element.get_attribute("outerHTML")

    # Use BeautifulSoup to parse the HTML content
    soup = bs(html_content, "html.parser")



    # Find each 'a' anchor tag matching the regex search below to find info for each school
    result_divs = soup.find_all('a', class_ = re.compile('SearchResult_root__'))

    # Loop over each result to extract name of school
    for div in result_divs:


        # School Name
        # Find name of school 
        name_heading = div.find_all('h2', class_ = re.compile('SearchResult_heading__'))
        name_pattern = r">.*?</"

        # Use regex to find the pattern in the name_heading and extract the matching text
        name = re.search(name_pattern, str(name_heading))
        
        # Extract all matches in a single group
        name = name.group()
        
        # Use string slicing to remove the ends that helped return the result
        name = name[1:-2]


        # School Address
        # Find & Collect each full address
        address_heading = div.find_all('div', class_ = re.compile('SearchResult_address__'))
        address_pattern  = r">.*?</"
        address_full = re.search(address_pattern, str(address_heading))
        address_full = address_full.group()

        # Trim result address string of regex remnants
        address_full = address_full[1:-2]


        try:
            # Extract different parts of address information
            address_detail_pattern  = re.compile(r"(.*(\s.*)*,)\s(.*,)\s(\w*,)\s(\d+-*\d*,|.{3}\s.{3},)\s(\w*)")  # regex expression
            address_detail = address_detail_pattern.search(address_full)   # search full address for patterns

            if address_detail != None:
                # Save each section of results to a different variable
                address_street, address_road_type, address_city, address_state, address_zip, address_country = address_detail.groups()  # extract each group from pattern

                # Use string slicing to remove the ends that helped return the result
                address_street = address_street[:-1]
                address_city = address_city[:-1]
                address_state = address_state[:-1]
                address_zip = address_zip[:-1]

            else: 
                # If there is no match, set default value
                address_street, address_road_type, address_city, address_state, address_zip, address_country = None

        
        except Exception as e:
            # Set a default value if any exceptions occur during regex search
            print(f"An error occurred: {e}")
            address_street, address_road_type, address_city, address_state, address_zip, address_country = None
        


        # School Type
        # Find school type
        school_heading = div.find_all('dd')

        # Extract school type
        school_pattern  = re.compile(r">(.*)</")  # regex expression
        school_type = re.search(school_pattern, str(school_heading))  # search for pattern
        school_type = school_type.group()   # extract result from group 

        school_type = school_type[1:-2]


        # School Website
        # Find link
        link_pattern  = re.compile('href="/schools/(.){6}"')
        link_partial = re.search(link_pattern, str(div))
        link_partial = link_partial.group()

        link_partial = link_partial[6:-1]

        link_root = 'https://www.adventisteducation.org'

        link_full = link_root + link_partial


        # School Description
        # Find description
        desc_full = div.find_all('div', class_ = re.compile('SearchResult_intro__'))
        
        # Check if there is a description or not
        if desc_full:

            # Convert matching result to a string and trim off excess
            desc_full = str(desc_full)
            desc_full = desc_full[40:-7]


        new_row = pd.DataFrame([{'School Name': name, 'Full Address': address_full, 'Street Address': address_street, 'City': address_city, 'State': address_state, 'Zip Code': address_zip, 'Country': address_country, 'School Type': school_type, 'Website': link_full, 'Description': desc_full}])
        #print(new_row)

        df = pd.concat([df, new_row], ignore_index=True)

display(df)




,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...
1,Bethany SDA Child Development Center,"714 Cedar St, Montgomery, AL, 36106-1002, USA",714 Cedar St,Montgomery,AL,36106-1002,USA,ECP,https://www.adventisteducation.org/schools/ANTP27,[]
2,Big Cove Christian Academy,"6354 Highway 431 S, Owens Cross Roads, AL, 357...",6354 Highway 431 S,Owens Cross Roads,AL,35763-9210,USA,PK-08,https://www.adventisteducation.org/schools/ANTG16,[]
3,Ephesus Academy Child Development Center,"829 McMillon Ave SW, Birmingham, AL, 35211-173...",829 McMillon Ave SW,Birmingham,AL,35211-1730,USA,ECP,https://www.adventisteducation.org/schools/ANTP2A,[]
4,Ephesus Junior Academy,"829 McMillon Ave SW, Birmingham, AL, 35211-173...",829 McMillon Ave SW,Birmingham,AL,35211-1730,USA,PK-10,https://www.adventisteducation.org/schools/ANTP45,[]
...,...,...,...,...,...,...,...,...,...,...
885,Windsor Adventist Elementary School,"5350 Haig Ave, Windsor, ON, N8T 1K8, Canada",5350 Haig Ave,Windsor,ON,N8T 1K8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6M90,[]
886,Greaves Adventist Academy,"2330 West Hill Ave, Montreal, QC, H4B 2S4, Canada",2330 West Hill Ave,Montreal,QC,H4B 2S4,Canada,PK-12,https://www.adventisteducation.org/schools/AN6P15,[]
887,Sartigan Adventist Academy,"645 7e Rue Sartigan, Saint-Georges, QC, G5Y 5B...",645 7e Rue Sartigan,Saint-Georges,QC,G5Y 5B8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6PAS,[]
888,Curtis-Horne Christian School,"3718 Hill Ave, Regina, SK, S4S 0X5, Canada",3718 Hill Ave,Regina,SK,S4S 0X5,Canada,K-09,https://www.adventisteducation.org/schools/AN6B20,[]


In [231]:
# Remember to close the browser
driver.quit()

# Verify Street Address

In [236]:
# Verify if each street address was correctly extracted from the full address
df['Addresses Match'] = df.apply(lambda row: row['Full Address'].startswith(row['Street Address']), axis=1)

In [240]:
df.head(1)

,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description,Addresses Match
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,True


In [238]:
# Count the number of False values returned (a mismatch between address values)
df['Addresses Match'].value_counts()

True    890
Name: Addresses Match, dtype: int64

We can see that there are zero False values in the 'Addresses Match' column. There should be no address mismatches between the 'Full Address' and 'Street Address', which was an issue earlier in the development. 

As such, we will drop the new column as it is no longer needed.

In [241]:
df.drop('Addresses Match', axis=1, inplace=True)
df.head(1)

,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...


# Export Results to File

In [230]:
# Export dataframe to a CSV file

# Specify file name and include index 
df.to_csv('Data/2024_NASDA_Education_processed.csv', index=True)

# Serialize Results to File

Use joblib to serialize data and variables.

In [235]:
# serialize results to a file: future

# create dictionary with variables to save for later
export = {
    'df': df
    }

# save dictionary as a file
joblib.dump(export, '2024_NASDA_Education_export')

# import joblib file with serialized variables
loaded_data = joblib.load('2024_NASDA_Education_export')

# display keys to use to save to new/active variables
loaded_data.keys()

dict_keys(['df'])

# Initial Summary

In this project, the goal was consolidating the different types and locations of Adventist schools across North America so that people considering moving to/within North America can find, *in one place*, what Adventist Education options are available where they might be relocating to.

The exported data above was imported into Tableau, where a few shortcomings were discovered.

> * First, Tableau was unable to process raw addresses
> * Second, some cleaning & formatting, of both previous data and imported Geocoded data, was needed to improve display and remove redundancy
> * Third, Tableau was unable to process individual grade levels without One-Hot Encoding that information

These issues have been addressed below.

# Geocoding & Grade Filtering

Geocoded addresses were processed using Geocod.io to extract Latitude & Longitude coordinates that Tableau can use. The data was imported and cleaned.

# Load the New Data

In [50]:
# Import the data
path = 'Data/2024_NASDA_Education_processed_geocodio.csv'

df_OE = pd.read_csv(path)

In [51]:
# Display first few rows of imported file
# Geocoding was provided by https://dash.geocod.io which added the additional rows at the end to the original file.
df_OE.head(3)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip,Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.372560,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery
1,1,Bethany SDA Child Development Center,"714 Cedar St, Montgomery, AL, 36106-1002, USA",714 Cedar St,Montgomery,AL,36106-1002,USA,ECP,https://www.adventisteducation.org/schools/ANTP27,[],32.371569,-86.289672,1.0,rooftop,714,Cedar St,NaN,NaN,Montgomery,AL,Montgomery County,36106,US,City of Montgomery
2,2,Big Cove Christian Academy,"6354 Highway 431 S, Owens Cross Roads, AL, 357...",6354 Highway 431 S,Owens Cross Roads,AL,35763-9210,USA,PK-08,https://www.adventisteducation.org/schools/ANTG16,[],34.667724,-86.488204,1.0,range_interpolation,6354,US-431,NaN,NaN,Owens Cross Roads,AL,Madison County,35763,US,TIGER/Line® dataset from the US Census Bureau


In [115]:
# create copy in memory so dataframe does not need to be reimported to rerun cells below
dfx = df_OE.copy()

In [116]:
# verify column names and data types
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      890 non-null    int64  
 1   School Name     890 non-null    object 
 2   Full Address    890 non-null    object 
 3   Street Address  890 non-null    object 
 4   City            890 non-null    object 
 5   State           890 non-null    object 
 6   Zip Code        890 non-null    object 
 7   Country         890 non-null    object 
 8   School Type     890 non-null    object 
 9   Website         890 non-null    object 
 10  Description     890 non-null    object 
 11  Latitude        890 non-null    float64
 12  Longitude       890 non-null    float64
 13  Accuracy Score  890 non-null    float64
 14  Accuracy Type   890 non-null    object 
 15  Number          867 non-null    object 
 16  Street          878 non-null    object 
 17  Unit Type       9 non-null      obj

# Clean Data

## Rename Features for Clarity

In [117]:
dfx.head(1)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip Code,Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip,Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery


In [118]:
# Keep both Zip Code and Zip, but rename to clarify between the number of digits included.
dfx = dfx.rename(columns={"Zip Code": "Zip (full)", "Zip": "Zip (5-digit)"})
dfx.head(1)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip (5-digit),Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery


In [119]:
# Keep both Number, but rename to show that it is part of the separated items (Street & Street Number) from what was originally combined in Street Address
dfx = dfx.rename(columns={"Number": "Street Number"})
dfx.head(1)

,Unnamed: 0,School Name,Full Address,Street Address,City,State,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Street Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip (5-digit),Country.1,Source
0,0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1.0,rooftop,1765,Highland Ave,NaN,NaN,Montgomery,AL,Montgomery County,36107,US,City of Montgomery


In [120]:
# Rename State to State/Province to show that Canadian Provinces are also included in that feature.
dfx = dfx.rename(columns={"State": "State/Province"})
dfx.tail(3)

,Unnamed: 0,School Name,Full Address,Street Address,City,State/Province,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Accuracy Score,Accuracy Type,Street Number,Street,Unit Type,Unit Number,City.1,State.1,County,Zip (5-digit),Country.1,Source
887,887,Sartigan Adventist Academy,"645 7e Rue Sartigan, Saint-Georges, QC, G5Y 5B...",645 7e Rue Sartigan,Saint-Georges,QC,G5Y 5B8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6PAS,[],46.083817,-70.645087,0.9,point,645,7e Rue Sartigan,NaN,NaN,Saint-Georges,QC,NaN,G6A,CA,CanVec+ by Natural Resources Canada
888,888,Curtis-Horne Christian School,"3718 Hill Ave, Regina, SK, S4S 0X5, Canada",3718 Hill Ave,Regina,SK,S4S 0X5,Canada,K-09,https://www.adventisteducation.org/schools/AN6B20,[],50.427137,-104.635031,1.0,rooftop,3718,Hill Ave,NaN,NaN,Regina,SK,NaN,S4S,CA,Open Government License - City of Regina Conta...
889,889,Riverside Christian School,"2228 Herman Ave, Saskatoon, SK, S7M 0N3, Canada",2228 Herman Ave,Saskatoon,SK,S7M 0N3,Canada,K-09,https://www.adventisteducation.org/schools/AN6B35,Riverside Christian School is dedicated to ser...,52.102478,-106.676862,0.9,nearest_rooftop_match,2227,Herman Ave,NaN,NaN,Saskatoon,SK,NaN,S7M,CA,Saskatoon


## Drop Redundant or Unneeded Features

In [121]:
# Create an empty list of columns to be dropped
to_drop = []

# Add previous index row to the drop list
to_drop.append("Unnamed: 0")

# Add duplicated and/or not needed features added by geocod.io to the drop list
to_drop.extend(["Accuracy Score", "Accuracy Type", "Unit Type", "Unit Number", "City.1", "State.1", "Country.1", "Source"])

print(to_drop)

['Unnamed: 0', 'Accuracy Score', 'Accuracy Type', 'Unit Type', 'Unit Number', 'City.1', 'State.1', 'Country.1', 'Source']


In [122]:
# Drop specified columns
dfx = dfx.drop(columns=to_drop, axis=1)
dfx.head(1)

,School Name,Full Address,Street Address,City,State/Province,Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Street Number,Street,County,Zip (5-digit)
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,Montgomery,AL,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475,1765,Highland Ave,Montgomery County,36107


## Reorder the Columns

In [123]:
# Create a list that will hold the names of each column, in the new desired order.
new_order = ["School Name", "Full Address", "Street Address", "Street Number", "Street", "City", "County", "State/Province", "Zip (5-digit)", "Zip (full)", "Country", "School Type", "Website", "Description", "Latitude", "Longitude"]

# Rearrange the order of columns using .reindex()
dfx = dfx.reindex(columns=new_order)

display(dfx.head(1))

,School Name,Full Address,Street Address,Street Number,Street,City,County,State/Province,Zip (5-digit),Zip (full),Country,School Type,Website,Description,Latitude,Longitude
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,1765,Highland Ave,Montgomery,Montgomery County,AL,36107,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475


## Fix Description when No Value

In [124]:
# Description currently has "[]" as the "null" value and displays poorly in Tableau
# Replace "null" value with an empty string "" to improve its display in Tableau

# Count number of instances, currently
print(dfx["Description"].value_counts().head(1))  # 606 instances of "[]"

# Replacement
dfx["Description"] = dfx["Description"].replace("[]", "")

# Recount updated number of instances
print(dfx["Description"].value_counts().head(1))   # 606 instances of ""

[]    606
Name: Description, dtype: int64
    606
Name: Description, dtype: int64


## Fix JP-12 Value

In [125]:
# There is a single value that was found through exploring the data in Tableau that appears to be either a typo or an outlier. 
# We will change this value to a more common value that is either very similar or what it actually should have been, originally.

# in School Type, explore the value "JP-12" 
print(dfx['School Type'].value_counts().sort_values().head(7))  # JK is a much more common value and is similar or probably correct
print()

# Replacement
dfx['School Type'] = dfx['School Type'].replace('JP-12', 'JK-12')

# Verify update
print(dfx['School Type'].value_counts().sort_values().head(6))  # Successfully updated

10-12      1
K-8        1
JP-12      1
JK-12      1
College    3
K-10       6
JK-08      6
Name: School Type, dtype: int64

10-12      1
K-8        1
JK-12      2
College    3
K-10       6
JK-08      6
Name: School Type, dtype: int64


## Fix Leading Zero Inconsistency 
Two values do not have a leading zero in the single-digit number:
> * "K-8"
> * "9-12"

We will add a leading zero to each for consistency and future filter simplicity.

In [126]:
# Explore the value "K-8" 
print(dfx['School Type'].value_counts().sort_values())  
print()

# Replacement
dfx['School Type'] = dfx['School Type'].replace('K-8', 'K-08')

# Verify update
print(dfx['School Type'].value_counts().sort_values())    # Successfully updated

10-12           1
K-8             1
JK-12           2
College         3
K-10            6
JK-08           6
University     11
K-09           14
K-12           30
PK-12          34
9-12           44
PK-10          69
K-08           76
ECP           169
PK-08         424
Name: School Type, dtype: int64

10-12           1
JK-12           2
College         3
K-10            6
JK-08           6
University     11
K-09           14
K-12           30
PK-12          34
9-12           44
PK-10          69
K-08           77
ECP           169
PK-08         424
Name: School Type, dtype: int64


In [127]:
# Explore the value "9-12" 
print(dfx['School Type'].value_counts().sort_values())  
print()

# Replacement
dfx['School Type'] = dfx['School Type'].replace('9-12', '09-12')

# Verify update
print(dfx['School Type'].value_counts().sort_values())    # Successfully updated

10-12           1
JK-12           2
College         3
K-10            6
JK-08           6
University     11
K-09           14
K-12           30
PK-12          34
9-12           44
PK-10          69
K-08           77
ECP           169
PK-08         424
Name: School Type, dtype: int64

10-12           1
JK-12           2
College         3
K-10            6
JK-08           6
University     11
K-09           14
K-12           30
PK-12          34
09-12          44
PK-10          69
K-08           77
ECP           169
PK-08         424
Name: School Type, dtype: int64


# Fix Ampersand

Some School Names contain the text "&amp;" instead of an actual ampersand (&). Fix so the names display correctly.

In [128]:
# Find all instances of "&amp;" with an actual ampersand (&).
dfx[dfx["School Name"].str.contains('&', na=False)]["School Name"]

201          GMAA Early Childhood &amp; Care Center
304         Stepping Stones Daycare &amp; PreSchool
624    Lighthouse Early Learning &amp; Legacy Acade
779        Brite Beginnings Preschool &amp; Daycare
782     Cascade Christian Childcare &amp; Preschool
Name: School Name, dtype: object

In [129]:
# Replace all instances of "&amp;" with an actual ampersand (&).
dfx['School Name'] = dfx['School Name'].str.replace('&amp;', '&')

In [130]:
# Replacement successful. 
dfx[dfx["School Name"].str.contains('&', na=False)]["School Name"]

201          GMAA Early Childhood & Care Center
304         Stepping Stones Daycare & PreSchool
624    Lighthouse Early Learning & Legacy Acade
779        Brite Beginnings Preschool & Daycare
782     Cascade Christian Childcare & Preschool
Name: School Name, dtype: object

# Grade Filtering
It is currently not possible to filter by grade level in Tableau using the School Type text field (i.e. - "PK-08", "K-12", "ECP", etc). We will use One-Hot Encoding to specifically specify which grades are available at each school, preschool through the last year of high school (or college/university). While True and False can be used in Tableau, it is less cumbersome to represent True with the digit 1, and False with the digit 0. 

In [131]:
dfx.head(1)

,School Name,Full Address,Street Address,Street Number,Street,City,County,State/Province,Zip (5-digit),Zip (full),Country,School Type,Website,Description,Latitude,Longitude
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,1765,Highland Ave,Montgomery,Montgomery County,AL,36107,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.37256,-86.284475


In [132]:
# Display School Type column, along with new Grade columns
print_grade_cols()

,School Type
0,PK-08
1,ECP
2,PK-10
3,PK-12
4,University
5,K-08
6,09-12
7,K-12
8,K-10
9,College


In [133]:
# Preschool boolean column
# Name new column and values to match
grade_level = ['Preschool', 'J', 'P']

# Create new column Preschool with value True (represented by the integer 1) in all School Types with a "J" or "P". 
dfx[grade_level[0]] = dfx['School Type'].str.contains('|'.join(grade_level)).astype(int)

print_grade_cols()

,School Type,Preschool
0,PK-08,1
1,ECP,1
2,PK-10,1
3,PK-12,1
4,University,0
5,K-08,0
6,09-12,0
7,K-12,0
8,K-10,0
9,College,0


In [134]:
# Kindergarten boolean column
# Name new column and value to match
grade_level = ('Kindergarten', 'K')

# Create new column Kindergarten with value True in all School Types with a "K". 
dfx[grade_level[0]] = dfx['School Type'].str.contains(grade_level[1]).astype(int)

print_grade_cols()

,School Type,Preschool,Kindergarten
0,PK-08,1,1
1,ECP,1,0
2,PK-10,1,1
3,PK-12,1,1
4,University,0,0
5,K-08,0,1
6,09-12,0,0
7,K-12,0,1
8,K-10,0,1
9,College,0,0


In [135]:
# Grade boolean columns
# Extract all two-digit numbers from 'School Type'/grades column 
grades_extracted = dfx['School Type'].str.findall(r'\b\d{2}\b')


# Initialize columns for each grade from 1 to 12
for grade in range(1, 13):
    dfx[f'Grade {grade}'] = 0


# Loop over each row and update the grade columns
# Pull two-digit grade values from each School Type 
for idx, grades_list in grades_extracted.items():

    # Case: there is a range between two numbered grades (i.e. 9-12)
    if len(grades_list) == 2:

        # Set lowest and highest grades in School Type range
        lowest_grade = grades_list[0]
        highest_grade = grades_list[1]

        # Drop lead zeros from lowest/highest grade
        lowest_grade = drop_lead_zero(lowest_grade)
        highest_grade = drop_lead_zero(highest_grade)
        
        # Select columns from Grade x to Grade y and all columns in between, inclusively; then set values to 1 for True
        dfx.loc[idx, f'Grade {lowest_grade}':f'Grade {highest_grade}'] = 1

        continue # Skip to the next iteration without executing the rest of the loop


    # Case: there is a range from before Grade 1 to a higher Grade y (i.e. K-12)
    if len(grades_list) == 1:

        # Set lowest and highest grades in School Type range
        lowest_grade = '1'
        highest_grade = grades_list[0]

        # Drop lead zeros from highest grade
        highest_grade = drop_lead_zero(highest_grade)

        # Select columns from Grade x to Grade y and all columns in between, inclusively; then set values to 1 for True
        dfx.loc[idx, f'Grade {lowest_grade}':f'Grade {highest_grade}'] = 1

print_grade_cols()

,School Type,Preschool,Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12
0,PK-08,1,1,1,1,1,1,1,1,1,1,0,0,0,0
1,ECP,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PK-10,1,1,1,1,1,1,1,1,1,1,1,1,0,0
3,PK-12,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,University,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,K-08,0,1,1,1,1,1,1,1,1,1,0,0,0,0
6,09-12,0,0,0,0,0,0,0,0,0,0,1,1,1,1
7,K-12,0,1,1,1,1,1,1,1,1,1,1,1,1,1
8,K-10,0,1,1,1,1,1,1,1,1,1,1,1,0,0
9,College,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [136]:
# Post-Secondary boolean column
# Name new column and value to match
grade_level = ('University', 'College')

# Create new column University/College with value True in all School Types with a 'University' or 'College'. 
dfx['University/College'] = dfx['School Type'].str.contains('|'.join(grade_level)).astype(int)

print_grade_cols()

,School Type,Preschool,Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,University/College
0,PK-08,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
1,ECP,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PK-10,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0
3,PK-12,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
4,University,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,K-08,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0
6,09-12,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0
7,K-12,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0
8,K-10,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0
9,College,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [137]:
# All boolean column
# We will have an All Levels column to allow us to easily view all schools in Tableau, regardless of grade-levels

# Create new column All Levels set to the value True 
dfx['All Types'] = 1

print_grade_cols()

,School Type,Preschool,Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,University/College,All Types
0,PK-08,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1
1,ECP,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,PK-10,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1
3,PK-12,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
4,University,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
5,K-08,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1
6,09-12,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1
7,K-12,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
8,K-10,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1
9,College,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


We have used One-Hot Encoding to specifically specify which grades are available at each school, preschool through the last year of high school, and even college/university. Our updated dataframe is ready to be exported and used with Tableau, again.

In [138]:
# Check if data appears reasonable: it does.
display(dfx.head())
display(dfx.tail())

,School Name,Full Address,Street Address,Street Number,Street,City,County,State/Province,Zip (5-digit),Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Preschool,Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,University/College,All Types
0,Bethany Christian Academy,"1765 Highland Ave, Montgomery, AL, 36107-2657,...",1765 Highland Ave,1765,Highland Ave,Montgomery,Montgomery County,AL,36107,36107-2657,USA,PK-08,https://www.adventisteducation.org/schools/ANTP25,Bethany Christian Academy is a Christ-centered...,32.372560,-86.284475,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1
1,Bethany SDA Child Development Center,"714 Cedar St, Montgomery, AL, 36106-1002, USA",714 Cedar St,714,Cedar St,Montgomery,Montgomery County,AL,36106,36106-1002,USA,ECP,https://www.adventisteducation.org/schools/ANTP27,,32.371569,-86.289672,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Big Cove Christian Academy,"6354 Highway 431 S, Owens Cross Roads, AL, 357...",6354 Highway 431 S,6354,US-431,Owens Cross Roads,Madison County,AL,35763,35763-9210,USA,PK-08,https://www.adventisteducation.org/schools/ANTG16,,34.667724,-86.488204,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1
3,Ephesus Academy Child Development Center,"829 McMillon Ave SW, Birmingham, AL, 35211-173...",829 McMillon Ave SW,829,McMillon Ave SW,Birmingham,Jefferson County,AL,35211,35211-1730,USA,ECP,https://www.adventisteducation.org/schools/ANTP2A,,33.494778,-86.844501,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,Ephesus Junior Academy,"829 McMillon Ave SW, Birmingham, AL, 35211-173...",829 McMillon Ave SW,829,McMillon Ave SW,Birmingham,Jefferson County,AL,35211,35211-1730,USA,PK-10,https://www.adventisteducation.org/schools/ANTP45,,33.494778,-86.844501,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1


,School Name,Full Address,Street Address,Street Number,Street,City,County,State/Province,Zip (5-digit),Zip (full),Country,School Type,Website,Description,Latitude,Longitude,Preschool,Kindergarten,Grade 1,Grade 2,Grade 3,Grade 4,Grade 5,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,University/College,All Types
885,Windsor Adventist Elementary School,"5350 Haig Ave, Windsor, ON, N8T 1K8, Canada",5350 Haig Ave,5350,Haig Ave,Windsor,NaN,ON,N8T,N8T 1K8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6M90,,42.300516,-82.954741,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1
886,Greaves Adventist Academy,"2330 West Hill Ave, Montreal, QC, H4B 2S4, Canada",2330 West Hill Ave,NaN,NaN,Montreal,NaN,QC,H4B,H4B 2S4,Canada,PK-12,https://www.adventisteducation.org/schools/AN6P15,,45.463981,-73.623193,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
887,Sartigan Adventist Academy,"645 7e Rue Sartigan, Saint-Georges, QC, G5Y 5B...",645 7e Rue Sartigan,645,7e Rue Sartigan,Saint-Georges,NaN,QC,G6A,G5Y 5B8,Canada,PK-08,https://www.adventisteducation.org/schools/AN6PAS,,46.083817,-70.645087,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1
888,Curtis-Horne Christian School,"3718 Hill Ave, Regina, SK, S4S 0X5, Canada",3718 Hill Ave,3718,Hill Ave,Regina,NaN,SK,S4S,S4S 0X5,Canada,K-09,https://www.adventisteducation.org/schools/AN6B20,,50.427137,-104.635031,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1
889,Riverside Christian School,"2228 Herman Ave, Saskatoon, SK, S7M 0N3, Canada",2228 Herman Ave,2227,Herman Ave,Saskatoon,NaN,SK,S7M,S7M 0N3,Canada,K-09,https://www.adventisteducation.org/schools/AN6B35,Riverside Christian School is dedicated to ser...,52.102478,-106.676862,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1


In [139]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   School Name         890 non-null    object 
 1   Full Address        890 non-null    object 
 2   Street Address      890 non-null    object 
 3   Street Number       867 non-null    object 
 4   Street              878 non-null    object 
 5   City                890 non-null    object 
 6   County              844 non-null    object 
 7   State/Province      890 non-null    object 
 8   Zip (5-digit)       890 non-null    object 
 9   Zip (full)          890 non-null    object 
 10  Country             890 non-null    object 
 11  School Type         890 non-null    object 
 12  Website             890 non-null    object 
 13  Description         890 non-null    object 
 14  Latitude            890 non-null    float64
 15  Longitude           890 non-null    float64
 16  Preschoo

# Export Updated Results to New File

In [140]:
# Export updated dataframe to a new CSV file

# Specify file name and do not include the index for the CSV file 
dfx.to_csv('Data/2024_NASDA_Education_processed_Geo+Grades.csv', index=False)